# This is computing TP, TN, FP, FN and precision, recall for DT

In [ ]:

import xarray as xr
import numpy as np
import cartopy
from cartopy import crs as ccrs
import matplotlib 
matplotlib.rcParams["font.size"] = 12
from matplotlib import pyplot as plt
from os.path import join, exists
from os import mkdir
import scipy
import netCDF4
import sklearn
import sys
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator, LongitudeLocator)
%matplotlib inline
import matplotlib.path as mpath
import importlib.util

import warnings
import pickle

warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
import shap

from sklearn.metrics import confusion_matrix,recall_score,precision_score

import os
import matplotlib as mpl
import tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)
import MM_util_AI
import MM_utilplot
from matplotlib import cm 
spec = importlib.util.spec_from_file_location("MM_dataprepare", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/MM_dataprepare.py")
MM_dataprepare = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_dataprepare
spec.loader.exec_module(MM_dataprepare)

spec = importlib.util.spec_from_file_location("MM_utilblocking", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/MM_utilblocking.py")
MM_utilblocking = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_utilblocking
spec.loader.exec_module(MM_utilblocking)


   
            
with open("/scratch/hz1994/blocking/data_MMmodel/filepath.txt","r") as fi:
    for ln in fi:
        if ln.startswith("dimensionalized_filepath"):
            dim_path=ln.strip().split('\t')[1]
        if ln.startswith("nondimensionalized_filepath"):
            nondim_path=ln.strip().split('\t')[1]
        if ln.startswith("code_filepath"):
            code_path=ln.strip().split('\t')[1]            
        if ln.startswith("DGindex_filepath"):
            DGindex_path=ln.strip().split('\t')[1]  
        if ln.startswith("conditionedT_filepath" ):
            train_path=ln.strip().split('\t')[1]
        if ln.startswith("model_filepath" ):
            models_path=ln.strip().split('\t')[1]
        if ln.startswith("fig_filepath" ):
            fig_path=ln.strip().split('\t')[1] 
print(dim_path)
print(nondim_path)
print(code_path)
print(DGindex_path)
print(train_path)
print(models_path)
print(fig_path)
from sklearn.metrics import confusion_matrix


In [ ]:
Duration=7
cnnsize='normal'
data_amount='1000.0k'

# direct training

In [ ]:
DT_tnfpfntp=True
if DT_tnfpfntp:
    X=np.load("/scratch/hz1994/blocking/data_era5/test_data_1940-2022.npy")
    Ysparse=np.load("/scratch/hz1994/blocking/data_era5/test_labels_1940-2022_T%d.npy"%Duration)  
    Y=np.zeros((Ysparse.size,2)).astype(bool)
    Y[:,1][Ysparse==1]=True  #blocking
    Y[:,0][Ysparse==0]=True
    num_total=X.shape[0]
    seed=10
    ordered_ind=np.arange(num_total)
    np.random.seed(seed)
    np.random.shuffle(ordered_ind)
    num=10
    shuffled_ind=np.random.shuffle(ordered_ind)
    test_data_list=(np.array_split(ordered_ind, num))

    for i in range(10):# the 7th training-test set pair
        print(i)
        test_ind=test_data_list[i]
        test_data=X[test_ind]
        test_labels=Y[test_ind]
        for random_seed in range(10):
            tn_list=[]
            fp_list=[]
            fn_list=[]
            tp_list=[]
            path="/scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_%ddaysblocking/%s/"\
            %(Duration,cnnsize) +"random_initCNN_%d/%d/"%(random_seed,i)

            for epoch in range(11):
                weightpath=path+"cp-%04d.ckpt"%epoch
                model = MM_util_AI.make_model((18,90,3))
                model.load_weights(weightpath).expect_partial()

                y_pred = model.predict(test_data)
                y_pred = np.argmax(y_pred, axis=1)
                tn, fp, fn, tp = confusion_matrix(test_labels[:,1], y_pred).ravel()
                tn_list.append(tn)
                fp_list.append(fp)
                fn_list.append(fn)
                tp_list.append(tp)
            np.save(path+"TN.npy",tn_list)
            np.save(path+"FP.npy",fp_list)
            np.save(path+"FN.npy",fn_list)
            np.save(path+"TP.npy",tp_list)

# For DT, Compute precision and recall by TP,FP,FN,TP above

In [ ]:
DT_precision_recall=True
if DT_precision_recall:
    for random_seed in range(10):
        precision_direct=[]
        recall_direct=[]
        print("random_seed=",random_seed)
        for i in range(10):# the 7th training-test set pair
            path="/scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_%ddaysblocking/%s/"\
            %(Duration,cnnsize) +"random_initCNN_%d/%d/"%(random_seed,i)

            tn_list=np.load(path+"TN.npy")
            fp_list=np.load(path+"FP.npy")
            fn_list=np.load(path+"FN.npy")
            tp_list=np.load(path+"TP.npy")
            recall_list=tp_list/(tp_list+fn_list) 
            precision_list=tp_list/(tp_list+fp_list)
            precision_direct.append(precision_list)
            recall_direct.append(recall_list)
        np.save("/scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_%ddaysblocking/%s/"\
            %(Duration,cnnsize) +"random_initCNN_%d/"%(random_seed)+"precision_direct.npy",precision_direct)
        np.save("/scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN/extreme_%ddaysblocking/%s/"\
            %(Duration,cnnsize) +"random_initCNN_%d/"%(random_seed)+"recall_direct.npy",recall_direct)